In [1]:
import os
import pandas as pd
import re
from datetime import datetime
import glob
import numpy as np

In [2]:
def process_bmkg_excel(file_path):
    """
    Membaca file Excel BMKG, menghapus 7 baris teratas dan baris bawah setelah data,
    dan memisahkan tanggal menjadi kolom Date, Year, Month, Day.
    
    Args:
        file_path: Path ke file Excel BMKG
        
    Returns:
        DataFrame hasil proses
    """
    print(f"Memproses file: {file_path}")
    
    # Baca Excel tanpa header
    df = pd.read_excel(file_path, header=None)
    
    # Cari indeks baris yang berisi header kolom (biasanya baris ke-7, indeks 6)
    header_row_idx = None
    for i in range(10):  # Cek 10 baris pertama
        row = df.iloc[i].astype(str)
        if "TANGGAL" in row.values:
            header_row_idx = i
            break
    
    if header_row_idx is None:
        print(f"Warning: Header tidak ditemukan di file {file_path}")
        return None
    
    # Gunakan baris header sebagai nama kolom dan hapus baris-baris sebelumnya
    headers = df.iloc[header_row_idx].tolist()
    df = df.iloc[header_row_idx+1:].reset_index(drop=True)
    df.columns = headers
    
    # Cari indeks baris terakhir sebelum keterangan
    last_row_idx = len(df)
    for i in range(len(df)):
        # Cek apakah baris berisi data valid atau hanya berisi nilai kosong/NaN
        row_data = df.iloc[i].dropna()
        if len(row_data) == 0 or (isinstance(df.iloc[i, 0], str) and ("KETERANGAN" in df.iloc[i, 0] or df.iloc[i, 0].strip() == "")):
            last_row_idx = i
            break
    
    # Ambil hanya data yang valid
    df = df.iloc[:last_row_idx]
    
    # Pastikan kolom TANGGAL ada
    if "TANGGAL" not in df.columns:
        print(f"Warning: Kolom TANGGAL tidak ditemukan di file {file_path}")
        return None
    
    # Konversi kolom TANGGAL ke datetime dan ekstrak Year, Month, Day
    # Format tanggal biasanya DD-MM-YYYY
    df['Date'] = pd.to_datetime(df['TANGGAL'], format='%d-%m-%Y', errors='coerce')
    
    # Jika konversi gagal, coba format lain
    if df['Date'].isna().all():
        df['Date'] = pd.to_datetime(df['TANGGAL'], errors='coerce')
    
    # Ekstrak komponen tanggal sebagai integer
    df['Year'] = df['Date'].dt.year.astype('Int64')  # Gunakan Int64 untuk mendukung nilai NaN
    df['Month'] = df['Date'].dt.month.astype('Int64')
    df['Day'] = df['Date'].dt.day.astype('Int64')
    
    # Reorganisasi kolom
    new_cols = ['Date', 'Year', 'Month', 'Day'] + [col for col in df.columns if col not in ['Date', 'Year', 'Month', 'Day', 'TANGGAL']]
    df = df[new_cols]
    
    # Ganti nilai 8888 dan 9999 dengan NaN
    df = df.replace([8888, 9999], pd.NA)
    
    # Konversi kolom numerik ke tipe data yang sesuai (tanpa desimal jika nilai selalu integer)
    for col in df.columns:
        if col not in ['Date', 'TANGGAL']:
            # Cek apakah kolom berisi nilai numerik
            if pd.api.types.is_numeric_dtype(df[col]):
                # Cek apakah semua nilai non-NaN adalah integer
                non_na_values = df[col].dropna()
                if len(non_na_values) > 0:
                    # Jika semua nilai adalah bilangan bulat, konversi ke integer
                    if all(non_na_values == non_na_values.astype(int)):
                        df[col] = df[col].astype('Int64')  # Int64 untuk mendukung nilai NaN
    
    # Hapus baris yang seluruhnya NaN
    df = df.dropna(how='all')
    
    return df

In [3]:
def process_year_directory(year_dir):
    """
    Memproses semua file Excel dalam direktori tahun tertentu
    dan menggabungkannya menjadi satu DataFrame.
    
    Args:
        year_dir: Path ke direktori tahun
        
    Returns:
        DataFrame gabungan dari semua file dalam direktori
    """
    print(f"Memproses direktori: {year_dir}")
    
    # Cari semua file Excel di direktori
    excel_files = glob.glob(os.path.join(year_dir, "*.xlsx"))
    
    if not excel_files:
        print(f"Tidak ada file Excel di direktori {year_dir}")
        return None
    
    # Proses semua file dan gabungkan hasilnya
    dfs = []
    for file_path in excel_files:
        df = process_bmkg_excel(file_path)
        if df is not None and not df.empty:
            dfs.append(df)
    
    if not dfs:
        print(f"Tidak ada data valid di direktori {year_dir}")
        return None
    
    # Gabungkan semua DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Urutkan berdasarkan tanggal
    combined_df = combined_df.sort_values(by='Date').reset_index(drop=True)
    
    return combined_df

In [4]:
def main():
    # Direktori induk yang berisi subfolder tahun
    parent_dir = '/run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh'
    
    # Output direktori (folder CSV di dalam direktori induk)
    output_dir = os.path.join(parent_dir, "CSV")
    os.makedirs(output_dir, exist_ok=True)
    
    # Cari semua subfolder tahun
    year_dirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d)) and d.isdigit()]
    
    for year in sorted(year_dirs):
        year_path = os.path.join(parent_dir, year)
        combined_df = process_year_directory(year_path)
        
        if combined_df is not None and not combined_df.empty:
            # Simpan sebagai CSV
            output_file = os.path.join(output_dir, f"BMKG_Data_{year}.csv")
            combined_df.to_csv(output_file, index=False)
            print(f"File CSV untuk tahun {year} telah dibuat: {output_file}")
            print(f"Jumlah baris data: {len(combined_df)}")
        
        print("-" * 50)

In [5]:
if __name__ == "__main__":
    main()

Memproses direktori: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428145920.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428145928.xlsx


/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428145936.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428145945.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428145952.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428145959.xlsx


/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428150006.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428150015.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428150030.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428150048.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2005/laporan_iklim_harian-250428150058.xlsx


...

/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/laporan_iklim_harian-250428161934.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/laporan_iklim_harian-250428161940.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/laporan_iklim_harian-250428161947.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/laporan_iklim_harian-250428161953.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/laporan_iklim_harian-250428162000.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/laporan_iklim_harian-250428162029.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2024/lap

/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

File CSV untuk tahun 2024 telah dibuat: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/CSV/BMKG_Data_2024.csv
Jumlah baris data: 366
--------------------------------------------------
Memproses direktori: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2025
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2025/laporan_iklim_harian-250428162216.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2025/laporan_iklim_harian-250428162226.xlsx
Memproses file: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/2025/laporan_iklim_harian-250428162231.xlsx
File CSV untuk tahun 2025 telah dibuat: /run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/BMKG/Stasiun Klimatologi Aceh/CSV/BMKG_Data_2025.csv
Jumlah baris data: 90
--------------------------------------------

/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
/tmp/ipykernel_176642/55253485.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([8888, 9999], pd.NA)
